# CNN Predictor for Cognitive Score

In [1]:
import pandas as pd
import numpy as np
from scipy.io import loadmat

# scikit-learn modules
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import tensorflow as tf
from tensorflow.keras import layers, models

import json
from joblib import dump

2024-04-16 12:38:15.241091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def normalise_correlate_fc(fc):
    fc_emp = fc / np.max(fc)
    fc_emp = np.corrcoef(fc_emp)
    return fc_emp

In [3]:
# evaluate 
def eval(model, x_test_scaled, y_test):
    y_pred = model.predict(x_test_scaled)
    r2_score = round(r2_score(y_test, y_pred),2)

    print(f'r2: {r2_score}')

In [4]:
ADSP_DATA = '../data/ADSP_PHC_COGN_Dec2023_FILTERED_wfiles.csv'

df_mem = pd.read_csv(ADSP_DATA)
df_exf = pd.read_csv(ADSP_DATA)
df_lan = pd.read_csv(ADSP_DATA)
df_vsp = pd.read_csv(ADSP_DATA)

df_mem = df_mem.drop(columns=['RID', 'VISCODE2', 'PHC_Diagnosis', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP'])
df_exf = df_exf.drop(columns=['RID', 'VISCODE2', 'PHC_Diagnosis', 'PHC_MEM', 'PHC_LAN', 'PHC_VSP'])
df_lan = df_lan.drop(columns=['RID', 'VISCODE2', 'PHC_Diagnosis', 'PHC_EXF', 'PHC_MEM', 'PHC_VSP'])
df_vsp = df_vsp.drop(columns=['RID', 'VISCODE2', 'PHC_Diagnosis', 'PHC_EXF', 'PHC_LAN', 'PHC_MEM'])
df_mem.shape

(1343, 3)

Try CNN for Memory

In [5]:
df_mem.dropna(subset=['PHC_MEM'], inplace=True)
df_mem.shape

(1342, 3)

In [7]:
train, test = train_test_split(df_mem, test_size=0.2, random_state=42)

# Get targets for training + testing each predictor
y_train, y_test = train['PHC_MEM'], test['PHC_MEM']

In [9]:
# Get features for training and testing

# Get the FC data as numpy arrays
dim_x = len(train)
x_train = []
x_test = []

for i, file in enumerate(train['FC_DATA'].values):
    arr = loadmat(f'../{file}')['ROI_activity'][:100, :] # get the first 100 regions
    fc = normalise_correlate_fc(arr)
    x_train.append(fc)

for i, file in enumerate(test['FC_DATA'].values):
    arr = loadmat(f'../{file}')['ROI_activity'][:100, :] # get the first 100 regions
    fc = normalise_correlate_fc(arr)
    x_test.append(fc)

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [11]:
x_train.shape

(1073, 100, 100)

## Simple CNN Model

In [12]:
from keras import backend as K

In [13]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [14]:
# Simple CNN architecture (32x64x64)

input_shape = (100,100,1)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# model = models.Sequential([
#     layers.Conv2D(8, (3, 3), activation='relu', input_shape=input_shape),
#     layers.BatchNormalization(), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(16, (3, 3), activation='relu'),
#     layers.BatchNormalization(), 
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(32, (3, 3), activation='relu'),
#     layers.BatchNormalization(), 
#     layers.Flatten(),
#     layers.Dense(32, activation='relu'),
#     layers.Dense(1) 
# ])

# model = models.Sequential()
# model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape = input_shape))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Flatten())

# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(32, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(32, activation='relu'))

# model.add(layers.Dense(1, activation='linear'))

2024-04-16 12:41:52.936161: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [15]:
model.compile(optimizer='adam', loss='mse', metrics=[r2_keras])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 32)        18464     
                                                                 
 flatten (Flatten)           (None, 14112)             0

In [16]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0005)

# history = model.fit(x_train, y_train, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stopping])
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
27/27 [==============================] - 5s 122ms/step - loss: 0.8247 - r2_keras: -0.2622 - val_loss: 0.8146 - val_r2_keras: -0.0118
Epoch 2/50
27/27 [==============================] - 3s 123ms/step - loss: 0.7160 - r2_keras: -0.0419 - val_loss: 0.8157 - val_r2_keras: -0.0169
Epoch 3/50
27/27 [==============================] - 3s 117ms/step - loss: 0.7131 - r2_keras: -0.0402 - val_loss: 0.8203 - val_r2_keras: -0.0256
Epoch 4/50
27/27 [==============================] - 3s 116ms/step - loss: 0.7078 - r2_keras: -0.0752 - val_loss: 0.9035 - val_r2_keras: -0.1306
Epoch 5/50
27/27 [==============================] - 3s 117ms/step - loss: 0.6934 - r2_keras: -0.0444 - val_loss: 0.8390 - val_r2_keras: -0.0399
Epoch 6/50
27/27 [==============================] - 3s 119ms/step - loss: 0.7293 - r2_keras: -0.0803 - val_loss: 0.8223 - val_r2_keras: -0.0266
Epoch 7/50
27/27 [==============================] - 3s 118ms/step - loss: 0.6919 - r2_keras: -0.0288 - val_loss: 0.7961 - val_r2_keras: 

In [17]:
loss, r2 = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test R2:", r2)

9/9 [==============================] - 0s 27ms/step - loss: 1.0152 - r2_keras: -0.5954
Test Loss: 1.0152337551116943
Test R2: -0.5954188108444214
